## OpenFDA PMA Queries

These functions will retreive information about PMA devices from OpenFDA.  You can see the list of fields and definitions on the [OpenFDA website](https://open.fda.gov/apis/device/pma/searchable-fields/).

In [1]:
import os
import random
import requests

api_key = 'IuSORFFEOUDgU0GmtCd0FcbgQige1Y7DDx26Hyna'

# Get the pma record for a particular supplement
def get_pma_supplement(pma_number, supplement_number):
    url = f'https://api.fda.gov/device/pma.json?api_key={api_key}&search=pma_number:{pma_number}+AND+supplement_number:{supplement_number}&limit=1'
    r = requests.get(url)
    if r.status_code != 200: 
        return {}
    else:
        return r.json()['results'][0]

# Get the original pma record
def get_pma_original(pma_number):
    url = f'https://api.fda.gov/device/pma.json?api_key={api_key}&search=pma_number:{pma_number}&sort=decision_date&limit=1'
    r = requests.get(url)
    if r.status_code != 200: 
        return {}
    else:
        return r.json()['results'][0]

### Get PMA Supplement

Fetches a particular PMA supplement.

In [2]:
get_pma_supplement('P820018','S001')

{'pma_number': 'P820018',
 'supplement_number': 'S001',
 'applicant': 'St. Jude Medical, Inc.',
 'street_1': '15900 VALLEY VIEW Court',
 'street_2': '',
 'city': 'SYLMAR',
 'state': 'CA',
 'zip': '91342',
 'zip_ext': '',
 'generic_name': 'Implantable pulse generator, pacemaker (non-CRT)',
 'trade_name': 'AUTIMA(TM) MODEL 2251 & MODEL 2600',
 'product_code': 'LWP',
 'advisory_committee': 'CV',
 'advisory_committee_description': 'Cardiovascular',
 'supplement_type': '',
 'supplement_reason': '',
 'expedited_review_flag': 'N',
 'date_received': '1983-02-22',
 'decision_date': '1983-06-06',
 'docket_number': '',
 'decision_code': 'APPR',
 'ao_statement': '',
 'openfda': {'registration_number': ['3004593495',
   '3004105270',
   '2135338',
   '2649622',
   '2124215',
   '3008483389',
   '3008159616',
   '3005334138',
   '3009220482',
   '9614453',
   '2032521',
   '3008973940',
   '3008494315',
   '3007566237',
   '2017865',
   '1450662',
   '3009380063',
   '2182208',
   '1000165971',
   '

### Get Original PMA Record
Fetches the original PMA record (i.e. S000)

In [3]:
get_pma_original('P820018')

{'pma_number': 'P820018',
 'supplement_number': '',
 'applicant': 'St. Jude Medical, Inc.',
 'street_1': '15900 VALLEY VIEW Court',
 'street_2': '',
 'city': 'SYLMAR',
 'state': 'CA',
 'zip': '91342',
 'zip_ext': '',
 'generic_name': 'Implantable pulse generator, pacemaker (non-CRT)',
 'trade_name': 'AUTIMA(TM) MODEL 2251 & MODEL 2600',
 'product_code': 'LWP',
 'advisory_committee': 'CV',
 'advisory_committee_description': 'Cardiovascular',
 'supplement_type': '',
 'supplement_reason': '',
 'expedited_review_flag': 'N',
 'date_received': '1982-03-30',
 'decision_date': '1983-02-10',
 'docket_number': '83M-0055',
 'fed_reg_notice_date': '1983-03-11',
 'decision_code': 'APWD',
 'ao_statement': '',
 'openfda': {'registration_number': ['3004593495',
   '3004105270',
   '2135338',
   '2649622',
   '2124215',
   '3008483389',
   '3008159616',
   '3005334138',
   '3009220482',
   '9614453',
   '2032521',
   '3008973940',
   '3008494315',
   '3007566237',
   '2017865',
   '1450662',
   '300938

### Extract Data
Extract a bunch of info out of the json.  Note that some devices to not have regulation numbers.

In [4]:
pma_data = get_pma_supplement('P820018','S001')
print(pma_data['advisory_committee_description'], 
      pma_data['openfda']['regulation_number'],
      pma_data['product_code'], 
      pma_data['generic_name'])

Cardiovascular  LWP Implantable pulse generator, pacemaker (non-CRT)


In [5]:
def cleanPMA(pma_data):
    
    openFDA = pma_data.pop("openfda")
    if "registration_number" in openFDA.keys():
        del openFDA["registration_number"]
    if "fei_number" in openFDA.keys():
        del openFDA["fei_number"]
    pma_data.update(openFDA)
    return pd.DataFrame(pma_data, index=[0])

In [6]:
import pandas as pd
ageDevices =pd.read_csv("ageAnnot.csv")

ageDevices[['PMANumber','Supplement']] = ageDevices.ID.str.split("_", expand=True)

uniqueIDs=ageDevices.ID.unique()

In [7]:
len(uniqueIDs)

284

In [10]:
#only 240 api calls allowed per minute

pma1=uniqueIDs[0].split("_")[0]
supp1=uniqueIDs[0].split("_")[1]

pma_data = get_pma_supplement(pma1, supp1)
pma=cleanPMA(pma_data)
noFind = []

for i in range(1, 240):
    pma1 = uniqueIDs[i].split("_")[0]
    supp1 = uniqueIDs[i].split("_")[1]
    if i%100 ==0:
        print(i)
    if supp1 == "S000":
        pma_data=get_pma_original(pma1)
        #print(pma1)
        pma2 = cleanPMA(pma_data)
        pma=pma.append(pma2, ignore_index = True)
        
    else:
        pma_data = get_pma_supplement(pma1,supp1)
        if pma_data=={}:
            print(pma1,supp1)
            noFind.append(i)
            print("cannot find")
        else:
            pma2 = cleanPMA(pma_data)
            pma=pma.append(pma2, ignore_index = True) 

P160029
P990056
P110039
P160015
P040001
P990027
P090022
P030053
P130016
P010018
P090026
P990080
P050034
P130007
P000027
P990013
P960058
P150024
P030040
P100044
P020028
P120010
P960019
P150021
P080028
P990078
P040039
P140021
P150019
100
P980040
P160026
P020025
P970032
P170037
P170018
P960036
P990014
P100018
P030008
P150040
P970034
P000016
P140029
P930014
P160012
P170013
P120021
P130019
P120023
P010068
P990025
P000015
P050020
P180008
P020050
P020056
P160019
P160034
P140015
P040008
P090018
P000029
P160017
P110033
P970053
P190016
P990055
P170002
P970056
P100033
200
P120005
P990010
P040026
P100034
P170039
P170024
P990052
P150034
P030031
P980031
P000021
P100026
P970043


In [11]:
#wait one minute then run this for remainder 

for i in range(240, len(uniqueIDs)):
    pma1 = uniqueIDs[i].split("_")[0]
    supp1 = uniqueIDs[i].split("_")[1]
    print(i)
    
    if supp1 == "S000":
        pma_data=get_pma_original(pma1)
        pma2 = cleanPMA(pma_data)
        pma=pma.append(pma2, ignore_index = True)
        
    else:
        pma_data = get_pma_supplement(pma1,supp1)
        if pma_data=={}:
            print(pma1,supp1)
            noFind.append(i)
            print("cannot find")
        else:
            pma2 = cleanPMA(pma_data)
            pma=pma.append(pma2, ignore_index = True) 

240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283


In [12]:
pma.to_csv("openFDAPMA.csv", index=False)